In [3]:
!pip install transformers torch torchvision scikit-learn pandas pillow tqdm openpyxl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 25.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.6 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.

In [2]:
!pip install ftfy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.1 MB/s eta 0:00:00


In [1]:
import sys
sys.path.append("/kaggle/input/dataset4")




import os
import argparse
from pathlib import Path
import pandas as pd
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import torchvision.transforms as T
import torchvision.models as models
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
from normalizer import normalize




class MemeDataset(Dataset):
    def __init__(self, df, images_dir, tokenizer, max_length=128, image_size=224, use_normalizer=True):
        self.df = df.reset_index(drop=True)
        self.images_dir = Path(images_dir)
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.image_size = image_size
        self.use_normalizer = use_normalizer


        self.transform = T.Compose([
            T.Resize((image_size, image_size)),
            T.ToTensor(),
            T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])


    def __len__(self):
        return len(self.df)


    def __getitem__(self, idx):
        row = self.df.loc[idx]
        img_path = self.images_dir / row['image_file_name']
        try:
            img = Image.open(img_path).convert('RGB')
        except Exception:
            img = Image.new('RGB', (self.image_size, self.image_size), color=(0, 0, 0))
        img = self.transform(img)


        text = str(row['text']) if pd.notna(row['text']) else ""
       
        # Normalize text using the normalizer
        if self.use_normalizer and text:
            try:
                text = normalize(text)
            except Exception as e:
                print(f"Warning: Normalization failed for text at index {idx}: {e}")
                # Fall back to original text if normalization fails
                pass
       
        tok = self.tokenizer(text, truncation=True, padding='max_length',
                             max_length=self.max_length, return_tensors='pt')
        input_ids = tok['input_ids'].squeeze(0)
        attention_mask = tok['attention_mask'].squeeze(0)


        label = int(row['label'])
        return {
            'image': img,
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'label': torch.tensor(label, dtype=torch.long)
        }




class MultimodalClassifier(nn.Module):
    def __init__(self, text_model_name='csebuetnlp/banglishbert',
                 num_labels=3, text_feat_dim=768, hidden_dim=512,
                 dropout=0.2, freeze_text=False, freeze_image=False):
        super().__init__()


        # TEXT ENCODER (BanglishBERT)
        self.text_encoder = AutoModel.from_pretrained(text_model_name)
        if freeze_text:
            for p in self.text_encoder.parameters():
                p.requires_grad = False


        # Determine BERT hidden size
        hidden_size = self.text_encoder.config.hidden_size


        self.text_proj = nn.Linear(hidden_size, text_feat_dim)


        
        # IMAGE ENCODER: ResNet-50
        # -------------------------------
# IMAGE ENCODER: ResNet-50
        # -------------------------------
        resnet = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        
        # Remove the final classification layer (fc)
        modules = list(resnet.children())[:-1]   # everything except last FC
        self.image_encoder = nn.Sequential(*modules)
        
        # Optional: freeze ResNet
        if freeze_image:
            for p in self.image_encoder.parameters():
                p.requires_grad = False
        
        # ResNet-50 feature size
        image_feat_dim = 2048
        self.image_proj = nn.Linear(image_feat_dim, image_feat_dim)



        # CLASSIFIER HEAD
        concat_dim = text_feat_dim + image_feat_dim
        self.classifier = nn.Sequential(
            nn.Linear(concat_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, num_labels),
        )


    def forward(self, input_ids, attention_mask, images):
        text_out = self.text_encoder(input_ids=input_ids, attention_mask=attention_mask)
        if hasattr(text_out, 'pooler_output') and text_out.pooler_output is not None:
            pooled = text_out.pooler_output
        else:
            last_hidden = text_out.last_hidden_state
            mask = attention_mask.unsqueeze(-1).float()
            summed = (last_hidden * mask).sum(1)
            denom = mask.sum(1).clamp(min=1e-9)
            pooled = summed / denom


        txt_feat = self.text_proj(pooled)


        #img_feat = self.image_encoder(images)  # [B, 1024]
        #img_feat = self.image_proj(img_feat)
        img_feat = self.image_encoder(images)        # [B, 2048, 1, 1]
        img_feat = img_feat.view(img_feat.size(0), -1)   # [B, 2048]
        img_feat = self.image_proj(img_feat)



        feat = torch.cat([txt_feat, img_feat], dim=1)
        logits = self.classifier(feat)
        return logits




def find_discrepancies(df, images_dir):
    images_dir = Path(images_dir)
    referenced = set(df['image_file_name'].astype(str).tolist())
    actual = set([p.name for p in images_dir.glob('*') if p.is_file()])
    missing = sorted(list(referenced - actual))
    orphan = sorted(list(actual - referenced))
    return missing, orphan




def prepare_dataframe(path, images_dir, drop_label_value=2):
    df = pd.read_excel(path)
    assert 'image_file_name' in df.columns and 'text' in df.columns and 'label' in df.columns, \
        "metadata.xlsx must contain columns: image_file_name, text, label"


    df = df[df['label'] != drop_label_value].copy()
    df['image_file_name'] = df['image_file_name'].astype(str).str.strip()


    missing, orphan = find_discrepancies(df, images_dir)
    if missing:
        print(f"Missing images for {len(missing)} metadata entries")
        df = df[~df['image_file_name'].isin(missing)].copy()


    if orphan:
        print(f"Found {len(orphan)} orphan image files not in metadata:")
        for o in orphan[:20]:
            print("  -", o)
        if len(orphan) > 20:
            print("  ... and more")


    unique_labels = sorted(df['label'].unique().tolist())
    label_map = {orig: idx for idx, orig in enumerate(unique_labels)}
    df['label'] = df['label'].map(label_map)
    print("Label mapping:", label_map)
    return df, orphan, label_map




def compute_class_weights(df):
    counts = df['label'].value_counts().sort_index().values
    weights = 1.0 / counts
    sample_weights = df['label'].map(lambda x: weights[x]).values
    return sample_weights




def collate_fn(batch):
    images = torch.stack([b['image'] for b in batch])
    input_ids = torch.stack([b['input_ids'] for b in batch])
    attention_mask = torch.stack([b['attention_mask'] for b in batch])
    labels = torch.stack([b['label'] for b in batch])
    return {
        'image': images,
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': labels
    }




def train_one_epoch(model, dataloader, optimizer, device):
    model.train()
    total_loss = 0.0
    criterion = nn.CrossEntropyLoss()
    for batch in tqdm(dataloader, desc="Train"):
        images = batch['image'].to(device)
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)


        optimizer.zero_grad()
        logits = model(input_ids=input_ids, attention_mask=attention_mask, images=images)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()


        total_loss += loss.item() * images.size(0)
    return total_loss / len(dataloader.dataset)




@torch.no_grad()
def evaluate(model, dataloader, device, label_map):
    model.eval()
    preds = []
    trues = []
    for batch in tqdm(dataloader, desc="Eval"):
        images = batch['image'].to(device)
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)


        logits = model(input_ids=input_ids, attention_mask=attention_mask, images=images)
        batch_preds = torch.argmax(logits, dim=1).cpu().numpy().tolist()
        batch_trues = labels.cpu().numpy().tolist()
        preds.extend(batch_preds)
        trues.extend(batch_trues)


    acc = accuracy_score(trues, preds)
    report = classification_report(trues, preds, digits=4)
    return acc, report, trues, preds




def main(args):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print("Device:", device)


    df, orphan_files, label_map = prepare_dataframe(args.data, args.images_dir, drop_label_value=2)


    if args.delete_orphans and orphan_files:
        for fname in orphan_files:
            p = Path(args.images_dir) / fname
            try:
                p.unlink()
            except Exception as e:
                print("Could not delete:", p, e)
        print("Deleted orphans.")


    train_df, test_df = train_test_split(df, test_size=args.test_size, stratify=df['label'], random_state=42)
    train_df, val_df = train_test_split(train_df, test_size=args.val_size, stratify=train_df['label'], random_state=42)
    print("Train / Val / Test sizes:", len(train_df), len(val_df), len(test_df))


    tokenizer = AutoTokenizer.from_pretrained(args.text_model)
   
    use_normalizer = not args.disable_normalizer
    if use_normalizer:
        print("Text normalization enabled")
    else:
        print("Text normalization disabled")
   
    train_dataset = MemeDataset(train_df, args.images_dir, tokenizer,
                                max_length=args.max_length, image_size=args.image_size,
                                use_normalizer=use_normalizer)
    val_dataset = MemeDataset(val_df, args.images_dir, tokenizer,
                              max_length=args.max_length, image_size=args.image_size,
                              use_normalizer=use_normalizer)
    test_dataset = MemeDataset(test_df, args.images_dir, tokenizer,
                               max_length=args.max_length, image_size=args.image_size,
                               use_normalizer=use_normalizer)


    sample_weights = compute_class_weights(train_df)
    sampler = WeightedRandomSampler(sample_weights, num_samples=len(train_dataset), replacement=True)


    train_loader = DataLoader(train_dataset, batch_size=args.batch_size, sampler=sampler, collate_fn=collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=args.batch_size, shuffle=False, collate_fn=collate_fn)
    test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False, collate_fn=collate_fn)


    num_labels = len(label_map)
    model = MultimodalClassifier(text_model_name=args.text_model,
                                 num_labels=num_labels,
                                 text_feat_dim=args.text_feat_dim,
                                 hidden_dim=args.hidden_dim,
                                 dropout=args.dropout,
                                 freeze_text=args.freeze_text,
                                 freeze_image=args.freeze_image)
    model.to(device)


    optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=args.lr)


    best_val_acc = 0.0
    os.makedirs(args.out_dir, exist_ok=True)


    for epoch in range(1, args.epochs + 1):
        print(f"Epoch {epoch}/{args.epochs}")
        train_loss = train_one_epoch(model, train_loader, optimizer, device)
        print("Train loss:", train_loss)
        val_acc, val_report, _, _ = evaluate(model, val_loader, device, label_map)
        print("Validation Acc:", val_acc)
        print("Validation report:\n", val_report)


        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save({'model_state_dict': model.state_dict(), 'label_map': label_map},
                       os.path.join(args.out_dir, "best_model.pt"))
            print("Saved best model.")


    print("Testing best model …")
    ckpt = torch.load(os.path.join(args.out_dir, "best_model.pt"), map_location=device)
    model.load_state_dict(ckpt['model_state_dict'])


    test_acc, test_report, trues, preds = evaluate(model, test_loader, device, label_map)
    print("Test Acc:", test_acc)
    print("Test report:\n", test_report)


    out = test_df.reset_index(drop=True).copy()
    out['pred_idx'] = preds
    inv_map = {v: k for k, v in label_map.items()}
    out['pred_orig'] = out['pred_idx'].map(inv_map)
    out.to_csv(os.path.join(args.out_dir, "test_predictions.csv"), index=False)
    print("Saved test predictions to", os.path.join(args.out_dir, "test_predictions.csv"))




if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('--data', type=str, default='/kaggle/input/dataset4/metadata.xlsx')
    parser.add_argument('--images_dir', type=str, default='/kaggle/input/dataset4/images')
    parser.add_argument('--out_dir', type=str, default='/kaggle/working/output')
    parser.add_argument('--epochs', type=int, default=30)
    parser.add_argument('--batch_size', type=int, default=16)
    parser.add_argument('--lr', type=float, default=2e-5)
    parser.add_argument('--text_model', type=str, default='csebuetnlp/banglishbert')
    parser.add_argument('--max_length', type=int, default=64)
    parser.add_argument('--image_size', type=int, default=224)
    parser.add_argument('--val_size', type=float, default=0.1)
    parser.add_argument('--test_size', type=float, default=0.1)
    parser.add_argument('--text_feat_dim', type=int, default=768)  # match BERT hidden
    parser.add_argument('--hidden_dim', type=int, default=512)
    parser.add_argument('--dropout', type=float, default=0.2)
    parser.add_argument('--freeze_text', action='store_true')
    parser.add_argument('--freeze_image', action='store_true')
    parser.add_argument('--delete-orphans', action='store_true')
    parser.add_argument('--disable-normalizer', action='store_true',
                        help='Disable text normalization (enabled by default)')


    args = parser.parse_args([])
    main(args)



Device: cuda
Found 4 orphan image files not in metadata:
  - FB_IMG_1751540473613.jpg
  - FB_IMG_1751739942837.jpg
  - FB_IMG_1754929300743.jpg
  - FB_IMG_1755921270397.jpg
Label mapping: {0: 0, 1: 1, 3: 2}
Train / Val / Test sizes: 5508 612 680
Text normalization enabled


/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

Train loss: 0.012383533300157928


Eval: 100%|██████████| 39/39 [00:18<00:00,  2.06it/s]


Validation Acc: 0.6584967320261438
Validation report:
               precision    recall  f1-score   support

           0     0.6138    0.5174    0.5615       172
           1     0.5862    0.5152    0.5484       132
           2     0.7009    0.7987    0.7466       308

    accuracy                         0.6585       612
   macro avg     0.6336    0.6104    0.6188       612
weighted avg     0.6517    0.6585    0.6518       612

Testing best model …


Eval: 100%|██████████| 43/43 [00:24<00:00,  1.72it/s]

Test Acc: 0.6411764705882353
Test report:
               precision    recall  f1-score   support

           0     0.5732    0.4712    0.5172       191
           1     0.5703    0.5000    0.5328       146
           2     0.6911    0.7959    0.7398       343

    accuracy                         0.6412       680
   macro avg     0.6116    0.5890    0.5966       680
weighted avg     0.6321    0.6412    0.6329       680

Saved test predictions to /kaggle/working/output/test_predictions.csv
